In [1]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import random
import pickle

In [2]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,each_file) for sentence in data_temp]
    return dataset

In [3]:
#initialising porterstemmer
ps=PorterStemmer()

In [4]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],polarity) for sentence,polarity in data]    
    return dataset    

In [5]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
        vocab.extend(sentence.split())
    return vocab

In [6]:
#execution
labeled_data=load_dataset()    #loading labeled sentences
random.shuffle(labeled_data)
vocab=set(get_words(labeled_data)) #getting words 
vocab=set([ps.stem(word) for word in vocab])   #stemming
#with open('vocab_unigram','w') as f:
#    json.dump(vocab,f)
#labeled_data=to_unigram(labeled_data)    #converting to unigram

In [7]:
#bigrams
print labeled_data[100]

(u'would work support engin shore cf analysi provid process work live project client', 'sli_pos')


In [8]:
print len(labeled_data)

201378


In [10]:
print len(vocab)

6493


In [11]:
def extract_features(document):       #features are bag of words. document is a list of words of a sentence 
    features = {}
    for word in vocab:
        features['contains(%s)' % word] = (word in document)
    return features        

In [12]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [20]:
#dividing into training and test data
train_set=train_test[:-10000]
test_set=train_test[-1000:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [16]:
with open('naive_bayes_unigram_model') as f:
    classifier=pickle.load(f)
nltk.classify.accuracy(classifier, test_set)

0.165

In [27]:
print classifier.show_most_informative_features(100)

Most Informative Features
       contains(horribl) = True              neg : pos    =   1957.1 : 1.0
       contains(terribl) = True              neg : neutra =   1565.5 : 1.0
          contains(bore) = True              neg : pos    =   1428.4 : 1.0
           contains(bad) = True              neg : pos    =   1254.9 : 1.0
      contains(unfortun) = True              neg : pos    =    657.3 : 1.0
          contains(hard) = True           sli_ne : pos    =    559.7 : 1.0
      contains(everyday) = True           sli_ne : pos    =    519.0 : 1.0
          contains(sick) = True              neg : pos    =    460.5 : 1.0
           contains(due) = True           sli_ne : pos    =    455.2 : 1.0
        contains(tediou) = True              neg : sli_po =    428.6 : 1.0
         contains(heavi) = True           sli_ne : pos    =    408.8 : 1.0
         contains(littl) = True           sli_ne : pos    =    395.1 : 1.0
         contains(great) = True              pos : neg    =    387.4 : 1.0

In [4]:
#save  the model
#with open('naive_bayes_unigram_model','w') as f:
#    pickle.dump(classifier,f)

In [2]:
#load the model
with open('naive_bayes_unigram_model') as f:
    classifier=pickle.load(f)

In [25]:
classifier.classify(test_set[0][0])

'neg'

In [21]:
dist = classifier.prob_classify(extract_features(sen.split()))
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931


In [18]:
max(1,2)

2

In [22]:
polarity=0
for label in dist.samples():
    print("%s: %f" % (label, dist.prob(label)))
    polarity=max(polarity,dist.prob(label))
print polarity

neg: 0.585660
sli_pos: 0.035267
sli_neg: 0.230902
neutral: 0.123239
pos: 0.024931
0.585660041903


In [14]:
labeled_data[0]

(u'fix time leav', 'neutral')